In [1]:
import os
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import sklearn
from textblob import TextBlob
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from wordcloud import WordCloud
import time

In [7]:
data = pd.read_csv('jurnal2.csv', sep=',', encoding='utf-8') 
data.head()

,web-scraper-order,web-scraper-start-url,vol,vol-href,title,title-href,abstraksi
0,1641093346-303,http://ijere.iaescore.com/index.php/IJERE/issu...,"Vol 9, No 1: March 2020",http://ijere.iaescore.com/index.php/IJERE/issu...,Why some religious individuals are not emotion...,http://ijere.iaescore.com/index.php/IJERE/arti...,"In two studies, we intend to investigate wheth..."
1,1641093177-244,http://ijere.iaescore.com/index.php/IJERE/issu...,"Vol 8, No 3: September 2019",http://ijere.iaescore.com/index.php/IJERE/issu...,The impact of parental involvement and educati...,http://ijere.iaescore.com/index.php/IJERE/arti...,The academic achievement of students in school...
2,1641093953-529,http://ijere.iaescore.com/index.php/IJERE/issu...,"Vol 10, No 3: September 2021",http://ijere.iaescore.com/index.php/IJERE/issu...,Student experience of using mobile devices for...,http://ijere.iaescore.com/index.php/IJERE/arti...,Mobile devices have been utilized as an emergi...
3,1641093502-362,http://ijere.iaescore.com/index.php/IJERE/issu...,"Vol 9, No 3: September 2020",http://ijere.iaescore.com/index.php/IJERE/issu...,Interaction monitoring model of logo counselin...,http://ijere.iaescore.com/index.php/IJERE/arti...,The purpose of this research is to develop the...
4,1641093291-286,http://ijere.iaescore.com/index.php/IJERE/issu...,"Vol 9, No 1: March 2020",http://ijere.iaescore.com/index.php/IJERE/issu...,Literation movement for leading schools: Best ...,http://ijere.iaescore.com/index.php/IJERE/arti...,The school literacy movement is the first step...


In [8]:
data = data.drop(data.columns[[0,1,2,3,4,5]], axis=1)
data.head()

,abstraksi
0,"In two studies, we intend to investigate wheth..."
1,The academic achievement of students in school...
2,Mobile devices have been utilized as an emergi...
3,The purpose of this research is to develop the...
4,The school literacy movement is the first step...


In [9]:
# Removing Punctuationtrain

import string
import re
string.punctuation

def remove_punctuation(txt):
    '''a function for removing punctuation'''
    text = txt
    text = re.sub('@[^\s]+','',text)
    text = re.sub('http://\S+|https://\S+', '', text)
    text = re.sub("#\S+", " ", text)
    text = re.sub('\[.*?\]', '', str(text))
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('  ', '', text)
    return text

data['clean'] = data['abstraksi'].apply(lambda x: remove_punctuation(x))
data.head()

,abstraksi,clean
0,"In two studies, we intend to investigate wheth...",In two studies we intend to investigate whethe...
1,The academic achievement of students in school...,The academic achievement of students in school...
2,Mobile devices have been utilized as an emergi...,Mobile devices have been utilized as an emergi...
3,The purpose of this research is to develop the...,The purpose of this research is to develop the...
4,The school literacy movement is the first step...,The school literacy movement is the first step...


In [10]:
# Casefolding

def casefolding(txt):
    text = str(txt).lower()
    return text

data['casefolding'] = data['clean'].apply(lambda x: casefolding(x))
data.head()

,abstraksi,clean,casefolding
0,"In two studies, we intend to investigate wheth...",In two studies we intend to investigate whethe...,in two studies we intend to investigate whethe...
1,The academic achievement of students in school...,The academic achievement of students in school...,the academic achievement of students in school...
2,Mobile devices have been utilized as an emergi...,Mobile devices have been utilized as an emergi...,mobile devices have been utilized as an emergi...
3,The purpose of this research is to develop the...,The purpose of this research is to develop the...,the purpose of this research is to develop the...
4,The school literacy movement is the first step...,The school literacy movement is the first step...,the school literacy movement is the first step...


In [11]:
#tokenizing
def tokenize(kalimat):
    text = word_tokenize(kalimat)
    return text

data['tokenize'] = data['casefolding'].apply(lambda x: tokenize(x))
data.head()

,abstraksi,clean,casefolding,tokenize
0,"In two studies, we intend to investigate wheth...",In two studies we intend to investigate whethe...,in two studies we intend to investigate whethe...,"[in, two, studies, we, intend, to, investigate..."
1,The academic achievement of students in school...,The academic achievement of students in school...,the academic achievement of students in school...,"[the, academic, achievement, of, students, in,..."
2,Mobile devices have been utilized as an emergi...,Mobile devices have been utilized as an emergi...,mobile devices have been utilized as an emergi...,"[mobile, devices, have, been, utilized, as, an..."
3,The purpose of this research is to develop the...,The purpose of this research is to develop the...,the purpose of this research is to develop the...,"[the, purpose, of, this, research, is, to, dev..."
4,The school literacy movement is the first step...,The school literacy movement is the first step...,the school literacy movement is the first step...,"[the, school, literacy, movement, is, the, fir..."


In [12]:
#filtering

def stopwords_removal(txt):
  filtering = stopwords.words('indonesian', 'english')
  x = []
  data = []
  def myFunc(x):
    if x in filtering:
      return False
    else:
      return True
  fit = filter(myFunc, txt)
  for x in fit:
    data.append(x)
  return data

data['stopwords'] = data['tokenize'].apply(lambda x: stopwords_removal(x))
data.head()

,abstraksi,clean,casefolding,tokenize,stopwords
0,"In two studies, we intend to investigate wheth...",In two studies we intend to investigate whethe...,in two studies we intend to investigate whethe...,"[in, two, studies, we, intend, to, investigate...","[in, two, studies, we, intend, to, investigate..."
1,The academic achievement of students in school...,The academic achievement of students in school...,the academic achievement of students in school...,"[the, academic, achievement, of, students, in,...","[the, academic, achievement, of, students, in,..."
2,Mobile devices have been utilized as an emergi...,Mobile devices have been utilized as an emergi...,mobile devices have been utilized as an emergi...,"[mobile, devices, have, been, utilized, as, an...","[mobile, devices, have, been, utilized, as, an..."
3,The purpose of this research is to develop the...,The purpose of this research is to develop the...,the purpose of this research is to develop the...,"[the, purpose, of, this, research, is, to, dev...","[the, purpose, of, this, research, is, to, dev..."
4,The school literacy movement is the first step...,The school literacy movement is the first step...,the school literacy movement is the first step...,"[the, school, literacy, movement, is, the, fir...","[the, school, literacy, movement, is, the, fir..."


In [15]:
#stemming
from nltk.stem import PorterStemmer 
from nltk.stem.snowball import SnowballStemmer

inggris = stopwords.words('english')
p_stemmer = PorterStemmer()
s_stemmer = SnowballStemmer(language='english')

In [16]:
def stemming2(text):
    return [p_stemmer.stem(word) for word in text if word not in inggris]

# round7 = lambda x: stemming(x)
data['stemming'] = data['stopwords'].apply(lambda x: stemming2(x))

In [17]:
data.head()

,abstraksi,clean,casefolding,tokenize,stopwords,stemming
0,"In two studies, we intend to investigate wheth...",In two studies we intend to investigate whethe...,in two studies we intend to investigate whethe...,"[in, two, studies, we, intend, to, investigate...","[in, two, studies, we, intend, to, investigate...","[two, studi, intend, investig, whether, spirit..."
1,The academic achievement of students in school...,The academic achievement of students in school...,the academic achievement of students in school...,"[the, academic, achievement, of, students, in,...","[the, academic, achievement, of, students, in,...","[academ, achiev, student, school, subject, var..."
2,Mobile devices have been utilized as an emergi...,Mobile devices have been utilized as an emergi...,mobile devices have been utilized as an emergi...,"[mobile, devices, have, been, utilized, as, an...","[mobile, devices, have, been, utilized, as, an...","[mobil, devic, util, emerg, learn, tool, thece..."
3,The purpose of this research is to develop the...,The purpose of this research is to develop the...,the purpose of this research is to develop the...,"[the, purpose, of, this, research, is, to, dev...","[the, purpose, of, this, research, is, to, dev...","[purpos, research, develop, clientcounselor, i..."
4,The school literacy movement is the first step...,The school literacy movement is the first step...,the school literacy movement is the first step...,"[the, school, literacy, movement, is, the, fir...","[the, school, literacy, movement, is, the, fir...","[school, literaci, movement, first, step, scie..."


In [20]:
data['clean'].head(10)

0    In two studies we intend to investigate whethe...
1    The academic achievement of students in school...
2    Mobile devices have been utilized as an emergi...
3    The purpose of this research is to develop the...
4    The school literacy movement is the first step...
5    This research aimed to identify trends on prom...
6    This study aimed to reveal preservice Turkish ...
7    Madrasah Diniyah is a very special Islamic edu...
8    The purpose of this study was to evaluate the ...
9    The aim of this study was to explore the predi...
Name: clean, dtype: object

In [21]:
data['casefolding'].head(10)

0    in two studies we intend to investigate whethe...
1    the academic achievement of students in school...
2    mobile devices have been utilized as an emergi...
3    the purpose of this research is to develop the...
4    the school literacy movement is the first step...
5    this research aimed to identify trends on prom...
6    this study aimed to reveal preservice turkish ...
7    madrasah diniyah is a very special islamic edu...
8    the purpose of this study was to evaluate the ...
9    the aim of this study was to explore the predi...
Name: casefolding, dtype: object

In [22]:
data['tokenize'].head(10)

0    [in, two, studies, we, intend, to, investigate...
1    [the, academic, achievement, of, students, in,...
2    [mobile, devices, have, been, utilized, as, an...
3    [the, purpose, of, this, research, is, to, dev...
4    [the, school, literacy, movement, is, the, fir...
5    [this, research, aimed, to, identify, trends, ...
6    [this, study, aimed, to, reveal, preservice, t...
7    [madrasah, diniyah, is, a, very, special, isla...
8    [the, purpose, of, this, study, was, to, evalu...
9    [the, aim, of, this, study, was, to, explore, ...
Name: tokenize, dtype: object

In [23]:
data['stopwords'].head(10)

0    [in, two, studies, we, intend, to, investigate...
1    [the, academic, achievement, of, students, in,...
2    [mobile, devices, have, been, utilized, as, an...
3    [the, purpose, of, this, research, is, to, dev...
4    [the, school, literacy, movement, is, the, fir...
5    [this, research, aimed, to, identify, trends, ...
6    [this, study, aimed, to, reveal, preservice, t...
7    [madrasah, diniyah, is, a, very, special, isla...
8    [the, purpose, of, this, study, was, to, evalu...
9    [the, aim, of, this, study, was, to, explore, ...
Name: stopwords, dtype: object

In [24]:
data['stemming'].head(10)

0    [two, studi, intend, investig, whether, spirit...
1    [academ, achiev, student, school, subject, var...
2    [mobil, devic, util, emerg, learn, tool, thece...
3    [purpos, research, develop, clientcounselor, i...
4    [school, literaci, movement, first, step, scie...
5    [research, aim, identifi, trend, promot, chall...
6    [studi, aim, reveal, preservic, turkish, langu...
7    [madrasah, diniyah, special, islam, educ, syst...
8    [purpos, studi, evalu, effect, use, educ, game...
9    [aim, studi, explor, predict, effect, hope, re...
Name: stemming, dtype: object